In [1]:
import sys
import os
import gzip
import re
import matplotlib.pyplot as plt
import numpy as np
import pysam
import Bio
import pandas as pd
from Bio import SeqIO


In [2]:
curpath = os.path.abspath(os.path.curdir) # зафиксируем папку

## Класс для получения данных

In [3]:
def GetSam(curpath):
    '''Get the sam files path, make a table with start posirion, sequence, CIGAR values, flags
        For input the path with files, for input a table'''
    
    files_sam_list = [x for x in os.listdir(curpath) if x.startswith("cart_")] # endswith - кончается на (можно так сделать для начала)
    path_sam_list = [os.path.join(curpath, x) for x in files_sam_list] # сделали путь ко всем sam файлам

# здесь будет цикл для перебора всех sam файлов
    data = pd.read_csv(path_sam_list[1], sep = '\t',header=None,usecols=[0,3,9,5,1,2])
    data.columns = ['Название рида','Flag','Ref','Начало рида', 'CIGAR','Рид']
    data = data.loc[data['Flag'] != 2048] # чистка по флагам
    
    return data
GetSam(curpath)

,Название рида,Flag,Ref,Начало рида,CIGAR,Рид
0,IonXpress_unpaired_ID6,16,UCE792_Claster_0_Contig1,54,136M,CCTGGTGATCTCTCCTGAAGGGGATAATGGGAGAGGAGCGCATTGC...
1,IonXpress_unpaired_ID44,0,UCE167_Claster_0_Contig1,355,8S72M,TCCCCCCATACAAACACATTGACAGACAGAGAGCAGCTGGACAGTG...
2,IonXpress_unpaired_ID78,0,UCE468_Claster_0_Contig1,335,124M,CCTGCAATCTCCTCTCCAAAGTCAGCAGGCGGACAGGCAGGCAGGA...
3,IonXpress_unpaired_ID236,16,UCE939_Claster_1_Contig1,93,34M1I79M14S,GCTTCTGAAGTCCGTGGAATTAGTTTATACTTTCGGGTTCTCTAGC...
4,IonXpress_unpaired_ID259,0,UCE705_Claster_0_Contig2,242,79M,AGTCAGGGAGGGAGTAGTGGATGCACTTAGTGCTCTCCACTCCCCC...
...,...,...,...,...,...,...
20728,IonXpress_unpaired_ID987329,16,UCE891_Claster_0_Contig1,1,31S48M4S,CAGACTTCTACATGCTTTGTAAGTAGGAAAACCTGCAAAATCGGCA...
20729,IonXpress_unpaired_ID987315,16,UCE727_Claster_0_Contig1,1,15S36M28S,TCTCTCTCTCTCTACCTCTCTCTCTCTCACTCTCTCTCTCTCTCTC...
20730,IonXpress_unpaired_ID987371,16,UCE1133_Claster_0_Contig1,145,131M,GTGCGTGTGCGGGGGAGTGTGTTTGTGCGTGCCTGCCAGCCGACCA...
20731,IonXpress_unpaired_ID987386,0,UCE26_Claster_1_Contig1,212,31S27M2I17M81S,ATTTCTTTTCCATCTTCCTCTGTGATCTGAGTGTGCCTGGCGCCCT...


## Класс для работы с CIGAR

In [4]:
class SAM(object):
    '''Класс для парсинга SAM файлов'''
    def __init__(self, name, Flag, ref_name, start, CIGAR, read): # задаем объект, где как атрибуты будут имя рида, флаг и тд
        self.name = name
        self.Flag = Flag
        self.ref_name = ref_name
        self.start = start
        self.CIGAR = CIGAR
        self.read = read
        
    def Ref(self): # Подгрузим соответствующий референс в виде списка букв
        with open('All_refs.fasta', 'r') as refs:
            line = refs.readline()
            while line:
                if self.ref_name in line: # Если название референса из атрибутов совпадает с названием строки в фаста файле с референсами, следующую строку записываем как референс
                    line = refs.readline()
                    ref = list(line)
                line = refs.readline()
        return ref
    
    def ReadySeq(self):
        '''For input the self object, for output the seq is ready for comparing with reference'''
        cigar = list(self.CIGAR.strip()) #сделали список
        # для начала нужно сигар разбить на блоки и переделать цифры в инт. По идее олжно получиться int(61) str(S) int(37) str(M)...
        # Каждый символ пытаюсь перевести в инт, обработка исключений на буквы
        for i in range(len(cigar)): #пробуем каждый символ списка сделать интом
            try:
                cigar[i] = int(cigar[i])
            except ValueError:
                cigar[i] = cigar[i]

        # Потом объединяю цифры в одно число.
        cigar_new = []
        for i in range(len(cigar)): # если символ - строка - записываем его отдельно
            if type(cigar[i]) == str:
                cigar_new.append(cigar[i])                                    
            try:
                if type(cigar[i]) == type(cigar[i+1]) == type(cigar[i+2]) : # если три символа подряд это инт, объединяем их
                    cigar_new.append(int(str(cigar[i])+str(cigar[i+1])+str(cigar[i+2])))
                if type(cigar[i]) == type(cigar[i+1]) and type(cigar[i]) != type(cigar[i+2]) and type(cigar[i-1]) ==str:  # если два символа подряд одного типа (такое может быть только с инт), объединяем их  
                    cigar_new.append(int(str(cigar[i])+str(cigar[i+1])))
                if type(cigar[i+1]) == str and type(cigar[i-1]) == str: # если один инт - тоже записываем его
                    cigar_new.append(cigar[i])
            except IndexError: # это нужно на случай окончания строки, list index out of range
                continue
                
        # удобнее сделать словарь
        numbers = cigar_new[::2] # список с числом нуклеотидов
        clipping_type = [] # список с типом совпадений
        
        for i in range(1, len(cigar_new), 2): 
            clipping_type.append(cigar_new[i]);

        cigar_dict = [] # делаем словарь
        for i in range(len(numbers)):
            cigar_dict.append((numbers[i],clipping_type[i]))
            
        # Теперь прогоняем сам seq по строке cigar, для каждой буквы свое действие
        nucl_number = 0 # сразу столбец из data
        seq = self.read
        for e in cigar_dict:
            if e[1] == 'S':
                seq = seq[:nucl_number] + 'N' * e[0] + seq[nucl_number + e[0]:]
            if e[1] == 'D':
                seq = seq[:nucl_number] + ' ' * e[0] + seq[nucl_number:]
            if e[1] == 'H':
                continue # это пока, может быть hard clipping не вырезается
            if e[1] == 'I':
                seq = seq[:nucl_number] + seq[nucl_number + e[0]:]
            nucl_number += e[0]
        seq = seq.replace('N', '') 
        new_seq = self.start * ' ' + seq

        return new_seq

## Делаем словарь для покрытия

In [5]:
def RefCover(read.Ref(), SAM_table(data)):
    '''Делает словарь покрытий по конкретному референсу и сборке.
    На вход: результат работы функции GetRef и WorkingReads, 
    на выходе список словарей с покрытием для каждого нуклеотида в референсе'''
    ref_cover_list = [] # большой словарь для всех нуклеотидов, по всей длине референса
    for i in range(len(референса) + 1):
        cover_dict = {} # для каждого нуклеотида
        cover_dict['A, нуклеотид № {0:.0f}'.format(i+1)] = 0
        cover_dict['C, нуклеотид № {0:.0f}'.format(i+1)] = 0
        cover_dict['G, нуклеотид № {0:.0f}'.format(i+1)] = 0
        cover_dict['T, нуклеотид № {0:.0f}'.format(i+1)] = 0
        ref_cover_list.append(cover_dict)

    for read in SAM_table(data):
        for nucl_number, nucl in enumerate(read):
                if nucl == 'A':
                    ref_cover_list[nucl_number]['A, нуклеотид № {0:.0f}'.format(nucl_number + 1)]  += 1
                if nucl == 'T':
                    ref_cover_list[nucl_number]['T, нуклеотид № {0:.0f}'.format(nucl_number + 1)]  += 1
                if nucl == 'G':
                    ref_cover_list[nucl_number]['G, нуклеотид № {0:.0f}'.format(nucl_number + 1)]  += 1
                if nucl == 'C':
                    ref_cover_list[nucl_number]['C, нуклеотид № {0:.0f}'.format(nucl_number + 1)]  += 1
                    
    # посчитаем покрытие для каждого нуклеотида
    cover_list = [] # большой словарь для всех нуклеотидов, по всей длине референса
    for i in range(len(GetRef) + 1):
        cover_dict = {} # для каждого нуклеотида
        cover_dict['A, нуклеотид № {0:.0f}'.format(i+1)] = 0
        cover_dict['C, нуклеотид № {0:.0f}'.format(i+1)] = 0
        cover_dict['G, нуклеотид № {0:.0f}'.format(i+1)] = 0
        cover_dict['T, нуклеотид № {0:.0f}'.format(i+1)] = 0
        cover_list.append(cover_dict)


    for nucl_number, nucl in enumerate(ref_cover_list):
        cover = 0
        for key, value in nucl.items():
            cover += value
        for key, value in nucl.items():    
            if key == 'A, нуклеотид № {:.0f}'.format(nucl_number + 1):
                if cover == 0:
                    cover_list[nucl_number][key] =  0
                else:
                    cover_list[nucl_number][key] =  value * 100 / cover
            if key == 'T, нуклеотид № {:.0f}'.format(nucl_number + 1):
                if cover == 0:
                    cover_list[nucl_number][key] =  0
                else:
                    cover_list[nucl_number][key] =  value * 100 / cover
            if key == 'G, нуклеотид № {:.0f}'.format(nucl_number + 1):
                if cover == 0:
                    cover_list[nucl_number][key] =  0
                else:
                    cover_list[nucl_number][key] =  value * 100 / cover
            if key == 'C, нуклеотид № {:.0f}'.format(nucl_number + 1):
                if cover == 0:
                    cover_list[nucl_number][key] =  0
                else:
                    cover_list[nucl_number][key] =  value * 100 / cover    
    return cover_list

SyntaxError: invalid syntax (3601556913.py, line 1)

In [6]:
# Посчитаем все существующие замены в данной сборке:
def AllChanges(GetRef, WorkingReads):
    '''Смотрим на все существующие замены в этой сборке для этой хромосомы.
        На вход берем референс и риды сборки. На выходе получаем список списков, для каждого рида свой список.
        Внутри два элемента, номер рида и номер нуклеотида с типом замены'''
    changing = [] # список замен
    for nucl_number, nucl in enumerate(GetRef): # для всех нуклеотидов референса
        for seq_number, seq in enumerate(WorkingReads): # для каждого рида сборки
            try:
                if seq[nucl_number] != ' ': # если нуклеотид рида существует в этой позиции
                    if nucl != seq[nucl_number]: # если нуклеотид референса не равен нуклеотиду рида
                        changing.append(['Рид№ {0:.0f}'.format(seq_number), 
                                         'Замена нуклеотида {0:s} на {1:s}, Номер нуклеотида {2:.0f}'.format(nucl, seq[nucl_number], nucl_number +1)])                  
            except IndexError:
                continue
    return changing

In [7]:
# Посчитаем количество ридов с заменами и сколько замен в каждом риде:
def ReadWithChanges(AllChanges):
    '''Считаем количество ридов с заменами и считаем число замен в риде.
        На вход подаем список списков из функции AllChanges.
        На выход список кортежей с номером рида и числом замен в этом риде'''
    read_with_change = [] # Риды с заменами
    for e in AllChanges: # Для каждого элемента из общего файла существующих замен:
        read_with_change.append(e[0]) # Добавляем сюда номер рида

    change = [] 
    for read in read_with_change:
        change.append(('В {0:s}'.format(read), read_with_change.count(read))) # Считаем сколько раз этот рид встретился в наборе всех замен
    read_with_change = list(set(change)) # Берем сет от этого набора:
    return read_with_change

In [8]:
# Посчитаем по позиции нуклеотида сколько каких там замен
def NuclWithChange(AllChanges):
    '''Считаем количество замен в каждом конретном нуклеотиде.
        На вход подаем список списков из функции AllChanges.
        На выход список кортежей с номером нуклеотида и типом замены и число подобых замен'''
    nucleotide_with_change = []
    for e in AllChanges:
        nucleotide_with_change.append(e[-1])

    change = []
    for read in nucleotide_with_change:
        change.append(('В {0:s}'.format(read), nucleotide_with_change.count(read)))

    nucleotide_with_change = list(set(change))
    return sorted(nucleotide_with_change, key=lambda a: a[0][-3:])

In [9]:
NuclWithChange(AllChanges(GetRef(curpath), WorkingReads(curpath)))

IndexError: list index out of range